In [1]:
import os
import glob
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

gic_path = '/home/kaurov/data/work/gic/'

In [2]:
cat = np.genfromtxt('data/80B/a=0.0905/halo_catalog_a0.0905.dat')

In [3]:


iii = np.where(cat[:,8] == np.sort(cat[:,8])[-1])[0][0]

#plt.figure(6)
#plt.hist(np.log10(cat[:,8]))


In [4]:
LargeBox = 80
SmallBox = 10
LLetter = 'B'
if LLetter=='B':
    Letter = -2 # B

iid = cat[iii,0]

CentX = cat[iii,1]
CentY = cat[iii,2]
CentZ = cat[iii,3]

Out_dir = '%010i_%03i_%03i%s'%(iid,SmallBox,LargeBox,LLetter)
print(Out_dir)
print(CentX, CentY, CentZ)

0001166094_010_080B
24.45529 79.87081 56.0932


In [5]:
setupinc = '''
        parameter (NumLowTop = ''' + str(int(LargeBox / 80 * 512)) + ''')
        parameter (NumLow = ''' + str(int(SmallBox / 80 * 512)) + ''')
        parameter (NumRef = 2)
C
C  Use these if you need a non-cubic grid
C
        parameter (NumLowX = NumLow)
        parameter (NumLowY = NumLow)
        parameter (NumLowZ = NumLow)
        parameter (NumLowTopX = NumLowTop)
        parameter (NumLowTopY = NumLowTop)
        parameter (NumLowTopZ = NumLowTop)

        parameter (DxShift = ''' + str(int((CentX - 0.5*SmallBox) / 80 * 512)) + ''')
        parameter (DyShift = ''' + str(int((CentY - 0.5*SmallBox) / 80 * 512)) + ''')
        parameter (DzShift = ''' + str(int((CentZ - 0.5*SmallBox) / 80 * 512)) + ''')
'''

In [6]:
setupf = '''
      subroutine InitParameters
      include 'src/cosmo.inc'
      include 'src/power.inc'
      include 'src/config.inc'

      dir = '''+"'"+Out_dir+"'"+'''
      jobname = 'rei'''+Out_dir+''''

      OmX = 0.2557
      OmB = 0.0479
      OmL = 0.6964
      OmN = 0
      h100 = 0.6814

      cell = 20.0/128
      enn = 0.9675
      sigma8 = 0.8285

      iseed = '''+str(Letter)+'''

      TFname = 'TF/lingerics_lcdmP.lic'

      info = 1

      return
      end
      '''

In [7]:
with open(gic_path+"setup.inc", "w") as text_file:
    text_file.write(setupinc)

with open(gic_path+"setup.f", "w") as text_file:
    text_file.write(setupf)



In [13]:
os.system("cd %s && mkdir %s"%(gic_path, Out_dir))
os.system("cd %s && source ~/.set_gcc && make"%(gic_path))
os.system("cd %s && source ~/.set_gcc && make utils"%(gic_path))
os.system("cd %s && ./bin/gic-low -d -m -b -save"%(gic_path))
os.system("cd %s && ./bin/gic2ifrit rei%s_lr_M.den -den"%(gic_path,Out_dir))


0

In [14]:
os.system("cd %s && ./bin/gic-high -d -m -b -multi -uni=2"%(gic_path))

0

In [15]:
os.system("cd %s && ./bin/gic2ifrit rei%s_mr2_M.den -den"%(gic_path,Out_dir))

0